
    # Comprehensive Benchmarking of Parallel QAOA Portfolio Optimization\n",
    \n",
    This notebook implements an even more systematic and in-depth benchmarking study for QAOA portfolio optimization. As Abdulmalek, a telecom engineer focused on raising quantum technology awareness, understanding these performance characteristics is vital.\n",
    \n",
    We will evaluate how varying key parameters impact computational time and quantum solution quality. This version significantly expands the parameter space to include `N` (problem size), `p` (QAOA depth), `max_iterations_optimizer` (classical optimizer effort), `num_parallel_runs` (multi-start runs), and **`transpilation_level`** (Qiskit's circuit optimization).\n",
    \n",
    **Important Note on Runtime:**\n",
    Given the extensive parameter ranges you've provided, this benchmark will explore **3456 unique scenarios** (6 N x 6 p x 6 max_iter x 4 num_parallel_runs x 4 transpilation_level). Each scenario involves multiple QAOA runs. This will result in an **extremely long execution time
    **, potentially spanning many hours or even days depending on your hardware. It is highly recommended to start with a **smaller subset of parameters** for initial testing and understanding (e.g., fewer values for each parameter) before running the full set.\n",
    \n",
    **Key components:**\n",
    - **Self-contained QAOA Optimization**: The single optimization run logic is now included directly within this notebook, handling transpilation.\n",
    - **`concurrent.futures.ProcessPoolExecutor`**: Used for parallel execution of multi-start runs.\n",
    - **Systematic Parameter Variation**: Nested loops will iterate through all specified ranges.\n",
    - **Robust Data Collection**: Results from each scenario will be collected into a Pandas DataFrame.\n",
    - **Comprehensive Metrics**: We'll track wall-clock time, best solution energy, approximation ratio (where applicable), number of successful runs, etc.\n",
    - **Automated Visualization**: Plots will be generated to illustrate trends from the vast dataset."

In [2]:
import numpy as np
import os
import time
import concurrent.futures
import pandas as pd
import matplotlib.pyplot as plt
import json
from qiskit import QuantumCircuit, transpile
from qiskit.primitives import Sampler
from qiskit.algorithms.optimizers import COBYLA, LBFGSB, SLSQP, SPSA
from qiskit.circuit import ParameterVector
# Import relevant Qokit components\n",
from qokit.qaoa_circuit_portfolio import get_parameterized_qaoa_circuit
from qokit.qaoa_objective_portfolio import get_qaoa_objective_qubo
from qokit.portfolio_optimization import portfolio_brute_force
# For analysis of optimal bitstring (if needed later)\n",
from qiskit_aer import AerSimulator # Use AerSimulator directly for flexibility\n"

ModuleNotFoundError: No module named 'qiskit.algorithms'

In [ ]:
"## 2. Problem Definition Function (with DWE)\n",
"\n",
"This function defines the Portfolio Optimization problem with Domain Wall Encoding. It generates random $\\mu$ (means) and $\\Sigma$ (covariance matrix) for a given $N$ (number of assets). It also calculates the classical brute-force optimal energy for smaller $N$, which is essential for computing the Approximation Ratio (AR)."

In [ ]:
def define_po_problem(N, K, q, lambda_sum, problem_seed=None):\n",
    \"\"\"\n",
    Defines the Portfolio Optimization problem with Domain Wall Encoding.\n",
    Generates random mu and Sigma for the given N.\n",
    Calculates classical brute-force solution for small N.\n",
    \"\"\"\n",
    if problem_seed is not None:\n",
        np.random.seed(problem_seed) # for reproducibility of problem instance definition\n",
    mu = np.random.uniform(0.05, 0.20, N)\n",
    Sigma = np.random.uniform(0.001, 0.015, (N, N))\n",
    Sigma = (Sigma + Sigma.T) / 2 \n",
    Sigma = Sigma + np.eye(N) * 0.005 # Ensure positive semi-definite\n",
    # --- DWE Transformation Coefficients ---\n",
    factor_J_obj = (2 * q) / (K**2) \n",
    factor_h_linear_obj = -1 / K\n",
    factor_h_diagonal_obj = q / (K**2) \n",
    J_coeffs_objective = {}\n",
    h_coeffs_objective = {}\n",
    for i in range(N):\n",
        for j in range(i + 1, N):\n",
            J_coeffs_objective[(i, j)] = factor_J_obj * Sigma[i, j]\n",
    for i in range(N):\n",
        h_coeffs_objective[i] = factor_h_linear_obj * mu[i] + factor_h_diagonal_obj * Sigma[i, i]\n",
    J_coeffs_total = {}\n",
    h_coeffs_total = {}\n",
    for (i, j), val in J_coeffs_objective.items():\n",
        J_coeffs_total[(i, j)] = val + 2 * lambda_sum\n",
    for i, val in h_coeffs_objective.items():\n",
        h_coeffs_total[i] = val - 5 * lambda_sum\n",
    po_problem = {\n",
        "N\": N,\n",
        K\": K,
        q\": q, \n",
        \"J\": J_coeffs_total, \n",
        \"h\": h_coeffs_total, \n",
        \"means\": mu,         \n",
        \"cov\": Sigma,        \n",
        \"q_orig\": q,         \n",
    best_portfolio = (None, None) \n",
    if N <= 20: # Threshold for classical brute-force feasibility\n",
        try:
            # Brute-force solver expects the original problem definition without DWE penalty\n",
            original_po_problem_for_brute_force = {\"N\":N, \"K\":K, \"q\":q, \"means\":mu, \"cov\":Sigma}\n",
            best_portfolio = portfolio_brute_force(original_po_problem_for_brute_force, return_bitstring=False)
            # print(f\"  Brute-force classical optimal energy for N={N}: {best_portfolio[0]:.6f}\")\n",
        except Exception as e:
            print(f\"  Brute-force calculation failed for N={N}: {e}. Approximation Ratio will be 'N/A'.\")\
    return po_problem, best_portfolio
# --- Define Global Constant Problem Parameters ---\n",
# As requested, these are fixed for all problem instances.\n",
GLOBAL_Q = 0.5 # Risk aversion parameter\n",
GLOBAL_LAMBDA_SUM = 0 # DWE-inspired penalty coefficient for sum constraint, now fixed at 0\n",

# Helper function to determine K based on N\n",
def get_K_for_N(N_val):
    return int(N_val * 0.4)

In [ ]:
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Self-Contained QAOA Single Optimization Function\n",
    "\n",
    "This function performs a single QAOA optimization run for a given initial parameter set and problem. It now incorporates the `transpilation_level` for circuit optimization. This function effectively replaces the need for the external `qaoa_parallel_optimizer_worker.py` for this notebook."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "def execute_single_qaoa_optimization(initial_point, po_problem_arg, p_val, max_iterations_optimizer_val, num_shots_simulator_val, transpilation_level_val, run_id):\n",
    "    \"\"\"\n",
    "    Performs a single QAOA optimization run.\n",
    "    Returns a dictionary of results including optimal energy, runtime, and success status.\n",
    "    \"\"\"\n",
    "    start_time = time.perf_counter()\n",
    "    optimal_energy = float('inf')\n",
    "    nfev = 0\n",
    "    success = False\n",
    "    message = \"\"\n",
    "    optimal_params = None\n",
    "\n",
    "    try:\n",
    "        # Define the QAOA objective function for the given problem and 'p'\n",
    "        qaoa_objective = get_qaoa_objective_qubo(po_problem_arg['J'], po_problem_arg['h'])\n",
    "\n",
    "        # Get the parameterized QAOA circuit\n",
    "        gamma, beta = ParameterVector('gamma', p_val), ParameterVector('beta', p_val)\n",
    "        qaoa_circuit = get_parameterized_qaoa_circuit(\n",
    "            po_problem_arg['J'], po_problem_arg['h'], p=p_val, initial_state=None, gamma=gamma, beta=beta\n",
    "        )\n",
    "        \n",
    "        # Transpile the circuit based on the transpilation_level\n",
    "        # We use AerSimulator as the basis for transpilation here, assuming it's the target backend\n",
    "        # If a real quantum device is targeted, pass that backend here.\n",
    "        aer_simulator = AerSimulator() # Instantiate a simulator for transpilation targeting\n",
    "        transpiled_circuit = transpile(qaoa_circuit, backend=aer_simulator, optimization_level=transpilation_level_val)\n",
    "\n",
    "        # Define the QAOA energy evaluation function for the optimizer\n",
    "        def qaoa_energy_fn(params):\n",
    "            nonlocal nfev\n",
    "            nfev += 1\n",
    "            try:\n",
    "                bound_circuit = transpiled_circuit.assign_parameters(params)\n",
    "                sampler = Sampler()\n",
    "                job = sampler.run(bound_circuit, shots=num_shots_simulator_val)\n",
    "                result = job.result()\n",
    "                quasi_dist = result.quasi_dists[0]\n",
    "                \n",
    "                # Calculate the expectation value\n",
    "                exp_val = 0\n",
    "                for bitstring, prob in quasi_dist.items():\n",
    "                    exp_val += qaoa_objective.evaluate_energy(bitstring) * prob\n",
    "                \n",
    "                return exp_val\n",
    "            except Exception as e:\n",
    "                print(f\"  [Run {run_id}] Error in energy evaluation: {e}\")\n",
    "                return float('inf') # Return high energy on error\n",
    "\n",
    "        # Choose and run the classical optimizer\n",
    "        # Using COBYLA as it's generally robust and good for quantum optimization\n",
    "        optimizer = COBYLA(maxiter=max_iterations_optimizer_val, tol=0.0001)\n",
    "        \n",
    "        # COBYLA uses a callback for iterations, we'll track nfev manually inside qaoa_energy_fn\n",
    "        result = optimizer.minimize(fun=qaoa_energy_fn, x0=np.array(initial_point))\n",
    "\n",
    "        optimal_energy = result.fun\n",
    "        optimal_params = result.x\n",
    "        success = result.success\n",
    "        message = result.message\n",
    "\n",
    "    except Exception as e:\n",
    "        message = f\"Optimization failed: {e}\"\n",
    "\n",
    "    end_time = time.perf_counter()\n",
    "    runtime = end_time - start_time\n",
    "\n",
    "    return {\n",
    "        'run_id': run_id,\n",
    "        'success': success,\n",
    "        'optimal_energy': optimal_energy,\n",
    "        'runtime_seconds': runtime,\n",
    "        'nfev': nfev,\n",
    "        'message': message,\n",
    "        'optimal_params': optimal_params\n",
    "    }"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. QAOA Multi-Start Optimization Scenario Function\n",
    "\n",
    "This function orchestrates the parallel multi-start optimization for a given set of `N`, `p`, `max_iterations_optimizer`, `num_parallel_runs`, and `transpilation_level`. It leverages `execute_single_qaoa_optimization` to run multiple starting points concurrently and then summarizes the best results."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "def run_qaoa_scenario(N_val, K_val, p_val, num_parallel_runs_val, max_iterations_optimizer_val, num_shots_simulator_val, transpilation_level_val, po_problem_arg, best_portfolio_arg):\n",
    "    print(f\"\\n--- Running Scenario: N={N_val}, K={K_val}, p={p_val}, max_iter={max_iterations_optimizer_val}, num_runs={num_parallel_runs_val}, transpile_lvl={transpilation_level_val} ---\")\n",
    "    \n",
    "    # Generate Initial Points for Multi-Start\n",
    "    initial_points = []\n",
    "    for _ in range(num_parallel_runs_val):\n",
    "        # QAOA parameters are typically gamma and beta\n",
    "        gammas_initial = np.random.uniform(0, 2 * np.pi, p_val)\n",
    "        betas_initial = np.random.uniform(0, np.pi, p_val)\n",
    "        initial_points.append(tuple(np.concatenate((gammas_initial, betas_initial))))\n",
    "\n",
    "    # Parallel Execution\n",
    "    start_overall_time = time.perf_counter()\n",
    "\n",
    "    worker_args = [\n",
    "        (initial_point, po_problem_arg, p_val, max_iterations_optimizer_val, num_shots_simulator_val, transpilation_level_val, i + 1)\n",
    "        for i, initial_point in enumerate(initial_points)\n",
    "    ]\n",
    "\n",
    "    all_results = []\n",
    "    # Use a ProcessPoolExecutor to leverage multiple CPU cores\n",
    "    with concurrent.futures.ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:\n",
    "        futures = [executor.submit(execute_single_qaoa_optimization, *args) for args in worker_args]\n",
    "        \n",
    "        for future in concurrent.futures.as_completed(futures):\n",
    "            try:\n",
    "                all_results.append(future.result())\n",
    "            except Exception as exc:\n",
    "                print(f'A worker generated an exception for scenario (N={N_val}, p={p_val}, max_iter={max_iterations_optimizer_val}, trans_lvl={transpilation_level_val}): {exc}')\n",
    "                all_results.append({'success': False, 'optimal_energy': float('inf'), 'runtime_seconds': 0, 'nfev': 0, 'message': str(exc), 'run_id': -1, 'optimal_params': None})\n",
    "\n",
    "    end_overall_time = time.perf_counter()\n",
    "    overall_duration = end_overall_time - start_overall_time\n",
    "\n",
    "    # Process and Summarize Results\n",
    "    best_overall_energy = float('inf')\n",
    "    best_overall_params = None\n",
    "    num_successful_runs = 0\n",
    "    total_nfev = 0\n",
    "    total_runtime_single_runs = 0\n",
    "\n",
    "    # Sort results by optimal energy to easily find the best one\n",
    "    all_results.sort(key=lambda x: x['optimal_energy'])\n",
    "\n",
    "    for result in all_results:\n",
    "        if result['success']:\n",
    "            num_successful_runs += 1\n",
    "            total_nfev += result['nfev']\n",
    "            total_runtime_single_runs += result['runtime_seconds']\n",
    "            if result['optimal_energy'] < best_overall_energy:\n",
    "                best_overall_energy = result['optimal_energy']\n",
    "                best_overall_params = result['optimal_params']\n",
    "\n",
    "    avg_nfev = total_nfev / num_successful_runs if num_successful_runs > 0 else np.nan\n",
    "    avg_runtime_single = total_runtime_single_runs / num_successful_runs if num_successful_runs > 0 else np.nan\n",
    "\n",
    "    approximation_ratio = None\n",
    "    if best_portfolio_arg[0] is not None and best_portfolio_arg[1] is not None:\n",
    "        E_min_classical = best_portfolio_arg[0]\n",
    "        E_max_classical = best_portfolio_arg[1]\n",
    "        # Avoid division by zero and ensure valid energy\n",
    "        if (E_max_classical - E_min_classical) != 0 and best_overall_energy != float('inf'):\n",
    "            # AR = (E_QAOA - E_min) / (E_max - E_min). AR closer to 0 is better.\n",
    "            approximation_ratio = (best_overall_energy - E_min_classical) / (E_max_classical - E_min_classical)\n",
    "        else:\n",
    "            approximation_ratio = np.nan # Cannot calculate if denominator is zero or QAOA failed\n",
    "\n",
    "    return {\n",
    "        'N': N_val,\n",
    "        'K': K_val,\n",
    "        'p': p_val,\n",
    "        'num_parallel_runs': num_parallel_runs_val,\n",
    "        'max_iterations_optimizer': max_iterations_optimizer_val,\n",
    "        'num_shots_simulator': num_shots_simulator_val,\n",
    "        'transpilation_level': transpilation_level_val,\n",
    "        'best_overall_energy': best_overall_energy,\n",
    "        'approximation_ratio': approximation_ratio,\n",
    "        'overall_duration_seconds': overall_duration,\n",
    "        'num_successful_runs': num_successful_runs,\n",
    "        'avg_nfev_per_run': avg_nfev,\n",
    "        'avg_runtime_per_run_seconds': avg_runtime_single,\n",
    "        'best_overall_params': best_overall_params # Store optimal params for later analysis if needed\n",
    "    }"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Benchmarking Configuration and Execution\n",
    "\n",
    "This section defines the full parameter space for the benchmarking study and orchestrates the nested loops to gather results across all combinations."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# --- Define Benchmarking Parameters ---\n",
    "N_values_to_test = [5, 8, 10, 15, 20, 22]\n",
    "p_values_to_test = [1, 10, 30, 60, 80, 100]\n",
    "max_iterations_optimizer_values = [50, 150, 300, 600, 1200, 2200]\n",
    "num_parallel_runs_values = [1, 2, 3, 4] # Now a varying parameter\n",
    "transpilation_levels_to_test = [0, 1, 2, 3] # New varying parameter\n",
    "num_shots_simulator_fixed = 256 # Fixed as requested\n",
    "\n",
    "all_benchmark_results = []\n",
    "problem_seed_counter = 0 # To ensure unique but reproducible problem instances for each N\n",
    "\n",
    "print(f\"### Starting Comprehensive QAOA Benchmarking ###\")\n",
    "print(f\"Global Q: {GLOBAL_Q}, Global Lambda Sum: {GLOBAL_LAMBDA_SUM}\")\n",
    "print(f\"Fixed Simulator shots per run: {num_shots_simulator_fixed}\")\n",
    "print(\"--- Iterating through N, p, max_iterations_optimizer, num_parallel_runs, and transpilation_level ---\")\n",
    "print(\"WARNING: This will take a very long time due to the large number of combinations (3456 scenarios).\")\n",
    "print(\"Consider reducing the parameter ranges for initial tests.\")\n",
    "print(\"---\" * 20)\n",
    "\n",
    "for N_test in N_values_to_test:\n",
    "    problem_seed_counter += 1\n",
    "    K_test = get_K_for_N(N_test)\n",
    "    \n",
    "    print(f\"\\n>>> Defining problem for N={N_test}, K={K_test} (Problem Seed: {problem_seed_counter})...\")\n",
    "    po_problem_current, best_portfolio_current = define_po_problem(N_test, K_test, GLOBAL_Q, GLOBAL_LAMBDA_SUM, problem_seed=problem_seed_counter)\n",
    "    \n",
    "    if best_portfolio_current[0] is not None:\n",
    "        print(f\"  Classical Optimal Energy (E_min): {best_portfolio_current[0]:.6f}\")\n",
    "    else:\n",
    "        print(f\"  Classical brute-force not calculated for N={N_test}.\")\n",
    "        \n",
    "    for p_test in p_values_to_test:\n",
    "        for max_iter_test in max_iterations_optimizer_values:\n",
    "            for num_runs_test in num_parallel_runs_values:\n",
    "                for trans_lvl_test in transpilation_levels_to_test:\n",
    "                    scenario_results = run_qaoa_scenario(\n",
    "                        N_test, K_test,\n",
    "                        p_test,\n",
    "                        num_runs_test,\n",
    "                        max_iter_test,\n",
    "                        num_shots_simulator_fixed,\n",
    "                        trans_lvl_test,\n",
    "                        po_problem_current,\n",
    "                        best_portfolio_current\n",
    "                    )\n",
    "                    all_benchmark_results.append(scenario_results)\n",
    "\n",
    "print(\"\\n\" + \"===" * 20)\n",
    "print(\"### All Benchmarking Scenarios Completed ###\")\n",
    "\n",
    "# Convert results to DataFrame and display\n",
    "results_df = pd.DataFrame(all_benchmark_results)\n",
    "pd.set_option('display.max_rows', None) # Display all rows\n",
    "pd.set_option('display.max_columns', None) # Display all columns\n",
    "print(\"\\n--- Overall Benchmarking Results Summary ---\")\n",
    "print(results_df.to_string())\n",
    "\n",
    "# Optionally save results to CSV\n",
    "# results_df.to_csv('qaoa_comprehensive_benchmarking_results.csv', index=False)\n",
    "# print(\"\\nResults saved to qaoa_comprehensive_benchmarking_results.csv\")"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Visualization and Analysis\n",
    "\n",
    "This section generates various plots to visualize the trends and insights from the collected benchmarking data. Given the five varying parameters, we will focus on creating insightful 2D plots by fixing other parameters to analyze specific relationships."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "%matplotlib inline\n",
    "\n",
    "print(\"\\n### Visualizing Benchmark Results ###\")\n",
    "\n",
    "# Define fixed values for plots where other parameters are varied\n",
    "fixed_num_runs_plot = num_parallel_runs_values[0]\n",
    "fixed_trans_lvl_plot = transpilation_levels_to_test[0]\n",
    "fixed_max_iter_plot = max_iterations_optimizer_values[0]\n",
    "fixed_p_plot = p_values_to_test[0]\n",
    "\n",
    "# --- Trend: Best Energy, AR, Duration vs. 'p' (fixed N, max_iter, num_runs, trans_lvl) ---\n",
    "print(f\"\\n--- Trends vs. 'p' (Fixed: num_runs={fixed_num_runs_plot}, max_iter={fixed_max_iter_plot}, trans_lvl={fixed_trans_lvl_plot}) ---\")\n",
    "for N_val in N_values_to_test:\n",
    "    df_filtered = results_df[\n",
    "        (results_df['N'] == N_val) &\n",
    "        (results_df['num_parallel_runs'] == fixed_num_runs_plot) &\n",
    "        (results_df['max_iterations_optimizer'] == fixed_max_iter_plot) &\n",
    "        (results_df['transpilation_level'] == fixed_trans_lvl_plot)\n",
    "    ]\n",
    "    if df_filtered.empty:\n",
    "        print(f\"No data for N={N_val} with current fixed parameters for p-plots.\")\n",
    "        continue\n",
    "    \n",
    "    fig, axes = plt.subplots(1, 3, figsize=(18, 5))\n",
    "    fig.suptitle(f'N={N_val}, Runs={fixed_num_runs_plot}, Max Iter={fixed_max_iter_plot}, Transpile Lvl={fixed_trans_lvl_plot}, Shots={num_shots_simulator_fixed}', fontsize=16)\n",
    "\n",
    "    axes[0].plot(df_filtered['p'], df_filtered['best_overall_energy'], marker='o', linestyle='-', color='blue')\n",
    "    axes[0].set_title('Best Overall Energy vs. p')\n",
    "    axes[0].set_xlabel('QAOA Layers (p)')\n",
    "    axes[0].set_ylabel('Energy')\n",
    "    axes[0].grid(True)\n",
    "\n",
    "    if not df_filtered['approximation_ratio'].isnull().all():\n",
    "        axes[1].plot(df_filtered['p'], df_filtered['approximation_ratio'], marker='o', linestyle='-', color='red')\n",
    "        axes[1].set_title('Approximation Ratio vs. p')\n",
    "        axes[1].set_xlabel('QAOA Layers (p)')\n",
    "        axes[1].set_ylabel('Approximation Ratio (AR)')\n",
    "        axes[1].grid(True)\n",
    "        axes[1].axhline(y=0, color='gray', linestyle='--', linewidth=0.8, label='Optimal AR = 0')\n",
    "        axes[1].legend()\n",
    "    else:\n",
    "        axes[1].text(0.5, 0.5, \"AR not available\", horizontalalignment='center', verticalalignment='center', transform=axes[1].transAxes, fontsize=12)\n",
    "        axes[1].set_title('Approximation Ratio vs. p')\n",
    "        axes[1].set_xlabel('QAOA Layers (p)')\n",
    "        axes[1].set_ylabel('Approximation Ratio (AR)')\n",
    "\n",
    "    axes[2].plot(df_filtered['p'], df_filtered['overall_duration_seconds'], marker='o', linestyle='-', color='green')\n",
    "    axes[2].set_title('Overall Duration vs. p')\n",
    "    axes[2].set_xlabel('QAOA Layers (p)')\n",
    "    axes[2].set_ylabel('Overall Duration (seconds)')\n",
    "    axes[2].grid(True)\n",
    "    \n",
    "    plt.tight_layout(rect=[0, 0.03, 1, 0.95])\n",
    "    plt.show()\n",
    "\n",
    "# --- Trend: Best Energy, AR, Duration vs. 'max_iterations_optimizer' (fixed N, p, num_runs, trans_lvl) ---\n",
    "print(f\"\\n--- Trends vs. 'max_iterations_optimizer' (Fixed: num_runs={fixed_num_runs_plot}, p={fixed_p_plot}, trans_lvl={fixed_trans_lvl_plot}) ---\")\n",
    "for N_val in N_values_to_test:\n",
    "    df_filtered = results_df[\n",
    "        (results_df['N'] == N_val) &\n",
    "        (results_df['num_parallel_runs'] == fixed_num_runs_plot) &\n",
    "        (results_df['p'] == fixed_p_plot) &\n",
    "        (results_df['transpilation_level'] == fixed_trans_lvl_plot)\n",
    "    ]\n",
    "    if df_filtered.empty:\n",
    "        print(f\"No data for N={N_val} with current fixed parameters for max_iter-plots.\")\n",
    "        continue\n",
    "\n",
    "    fig, axes = plt.subplots(1, 3, figsize=(18, 5))\n",
    "    fig.suptitle(f'N={N_val}, Runs={fixed_num_runs_plot}, p={fixed_p_plot}, Transpile Lvl={fixed_trans_lvl_plot}, Shots={num_shots_simulator_fixed}', fontsize=16)\n",
    "\n",
    "    axes[0].plot(df_filtered['max_iterations_optimizer'], df_filtered['best_overall_energy'], marker='o', linestyle='-', color='blue')\n",
    "    axes[0].set_title('Best Overall Energy vs. Max Iterations')\n",
    "    axes[0].set_xlabel('Max Iterations (classical optimizer)')\n",
    "    axes[0].set_ylabel('Energy')\n",
    "    axes[0].grid(True)\n",
    "\n",
    "    if not df_filtered['approximation_ratio'].isnull().all():\n",
    "        axes[1].plot(df_filtered['max_iterations_optimizer'], df_filtered['approximation_ratio'], marker='o', linestyle='-', color='red')\n",
    "        axes[1].set_title('Approximation Ratio vs. Max Iterations')\n",
    "        axes[1].set_xlabel('Max Iterations (classical optimizer)')\n",
    "        axes[1].set_ylabel('Approximation Ratio (AR)')\n",
    "        axes[1].grid(True)\n",
    "        axes[1].axhline(y=0, color='gray', linestyle='--', linewidth=0.8, label='Optimal AR = 0')\n",
    "        axes[1].legend()\n",
    "    else:\n",
    "        axes[1].text(0.5, 0.5, \"AR not available\", horizontalalignment='center', verticalalignment='center', transform=axes[1].transAxes, fontsize=12)\n",
    "        axes[1].set_title('Approximation Ratio vs. Max Iterations')\n",
    "        axes[1].set_xlabel('Max Iterations (classical optimizer)')\n",
    "        axes[1].set_ylabel('Approximation Ratio (AR)')\n",
    "\n",
    "    axes[2].plot(df_filtered['max_iterations_optimizer'], df_filtered['overall_duration_seconds'], marker='o', linestyle='-', color='green')\n",
    "    axes[2].set_title('Overall Duration vs. Max Iterations')\n",
    "    axes[2].set_xlabel('Max Iterations (classical optimizer)')\n",
    "    axes[2].set_ylabel('Overall Duration (seconds)')\n",
    "    axes[2].grid(True)\n",
    "    \n",
    "    plt.tight_layout(rect=[0, 0.03, 1, 0.95])\n",
    "    plt.show()\n",
    "\n",
    "# --- Trend: Best Energy, AR, Duration vs. 'N' (fixed p, max_iter, num_runs, trans_lvl) ---\n",
    "print(f\"\\n--- Trends vs. 'N' (Fixed: num_runs={fixed_num_runs_plot}, p={fixed_p_plot}, max_iter={fixed_max_iter_plot}, trans_lvl={fixed_trans_lvl_plot}) ---\")\n",
    "df_filtered_N_trend = results_df[\n",
    "    (results_df['num_parallel_runs'] == fixed_num_runs_plot) &\n",
    "    (results_df['p'] == fixed_p_plot) &\n",
    "    (results_df['max_iterations_optimizer'] == fixed_max_iter_plot) &\n",
    "    (results_df['transpilation_level'] == fixed_trans_lvl_plot)\n",
    "]\n",
    "\n",
    "if not df_filtered_N_trend.empty:\n",
    "    fig, axes = plt.subplots(1, 3, figsize=(18, 5))\n",
    "    fig.suptitle(f'p={fixed_p_plot}, Max Iterations={fixed_max_iter_plot}, Runs={fixed_num_runs_plot}, Transpile Lvl={fixed_trans_lvl_plot}, Shots={num_shots_simulator_fixed}', fontsize=16)\n",
    "\n",
    "    axes[0].plot(df_filtered_N_trend['N'], df_filtered_N_trend['best_overall_energy'], marker='o', linestyle='-', color='blue')\n",
    "    axes[0].set_title('Best Overall Energy vs. N')\n",
    "    axes[0].set_xlabel('Number of Assets (N)')\n",
    "    axes[0].set_ylabel('Energy')\n",
    "    axes[0].grid(True)\n",
    "    axes[0].set_xticks(N_values_to_test)\n",
    "\n",
    "    if not df_filtered_N_trend['approximation_ratio'].isnull().all():\n",
    "        axes[1].plot(df_filtered_N_trend['N'], df_filtered_N_trend['approximation_ratio'], marker='o', linestyle='-', color='red')\n",
    "        axes[1].set_title('Approximation Ratio vs. N')\n",
    "        axes[1].set_xlabel('Number of Assets (N)')\n",
    "        axes[1].set_ylabel('Approximation Ratio (AR)')\n",
    "        axes[1].grid(True)\n",
    "        axes[1].axhline(y=0, color='gray', linestyle='--', linewidth=0.8, label='Optimal AR = 0')\n",
    "        axes[1].legend()\n",
    "    else:\n",
    "        axes[1].text(0.5, 0.5, \"AR not available\", horizontalalignment='center', verticalalignment='center', transform=axes[1].transAxes, fontsize=12)\n",
    "        axes[1].set_title('Approximation Ratio vs. N')\n",
    "        axes[1].set_xlabel('Number of Assets (N)')\n",
    "        axes[1].set_ylabel('Approximation Ratio (AR)')\n",
    "    axes[1].set_xticks(N_values_to_test)\n",
    "\n",
    "    axes[2].plot(df_filtered_N_trend['N'], df_filtered_N_trend['overall_duration_seconds'], marker='o', linestyle='-', color='green')\n",
    "    axes[2].set_title('Overall Duration vs. N')\n",
    "    axes[2].set_xlabel('Number of Assets (N)')\n",
    "    axes[2].set_ylabel('Overall Duration (seconds)')\n",
    "    axes[2].grid(True)\n",
    "    axes[2].set_xticks(N_values_to_test)\n",
    "    \n",
    "    plt.tight_layout(rect=[0, 0.03, 1, 0.95])\n",
    "    plt.show()\n",
    "\n",
    "else:\n",
    "    print(f\"No data for N trend analysis with p={fixed_p_plot}, max_iter={fixed_max_iter_plot}, num_runs={fixed_num_runs_plot}, trans_lvl={fixed_trans_lvl_plot}.\")\n",
    "\n",
    "# --- Trend: Best Energy, AR, Duration vs. 'num_parallel_runs' (fixed N, p, max_iter, trans_lvl) ---\n",
    "print(f\"\\n--- Trends vs. 'num_parallel_runs' (Fixed: N={N_values_to_test[0]}, p={fixed_p_plot}, max_iter={fixed_max_iter_plot}, trans_lvl={fixed_trans_lvl_plot}) ---\")\n",
    "df_filtered_runs_trend = results_df[\n",
    "    (results_df['N'] == N_values_to_test[0]) &\n",
    "    (results_df['p'] == fixed_p_plot) &\n",
    "    (results_df['max_iterations_optimizer'] == fixed_max_iter_plot) &\n",
    "    (results_df['transpilation_level'] == fixed_trans_lvl_plot)\n",
    "]\n",
    "\n",
    "if not df_filtered_runs_trend.empty:\n",
    "    fig, axes = plt.subplots(1, 3, figsize=(18, 5))\n",
    "    fig.suptitle(f'N={N_values_to_test[0]}, p={fixed_p_plot}, Max Iterations={fixed_max_iter_plot}, Transpile Lvl={fixed_trans_lvl_plot}, Shots={num_shots_simulator_fixed}', fontsize=16)\n",
    "\n",
    "    axes[0].plot(df_filtered_runs_trend['num_parallel_runs'], df_filtered_runs_trend['best_overall_energy'], marker='o', linestyle='-', color='blue')\n",
    "    axes[0].set_title('Best Overall Energy vs. Num Parallel Runs')\n",
    "    axes[0].set_xlabel('Number of Parallel Runs')\n",
    "    axes[0].set_ylabel('Energy')\n",
    "    axes[0].grid(True)\n",
    "    axes[0].set_xticks(num_parallel_runs_values)\n",
    "\n",
    "    if not df_filtered_runs_trend['approximation_ratio'].isnull().all():\n",
    "        axes[1].plot(df_filtered_runs_trend['num_parallel_runs'], df_filtered_runs_trend['approximation_ratio'], marker='o', linestyle='-', color='red')\n",
    "        axes[1].set_title('Approximation Ratio vs. Num Parallel Runs')\n",
    "        axes[1].set_xlabel('Number of Parallel Runs')\n",
    "        axes[1].set_ylabel('Approximation Ratio (AR)')\n",
    "        axes[1].grid(True)\n",
    "        axes[1].axhline(y=0, color='gray', linestyle='--', linewidth=0.8, label='Optimal AR = 0')\n",
    "        axes[1].legend()\n",
    "    else:\n",
    "        axes[1].text(0.5, 0.5, \"AR not available\", horizontalalignment='center', verticalalignment='center', transform=axes[1].transAxes, fontsize=12)\n",
    "        axes[1].set_title('Approximation Ratio vs. Num Parallel Runs')\n",
    "        axes[1].set_xlabel('Number of Parallel Runs')\n",
    "        axes[1].set_ylabel('Approximation Ratio (AR)')\n",
    "    axes[1].set_xticks(num_parallel_runs_values)\n",
    "\n",
    "    axes[2].plot(df_filtered_runs_trend['num_parallel_runs'], df_filtered_runs_trend['overall_duration_seconds'], marker='o', linestyle='-', color='green')\n",
    "    axes[2].set_title('Overall Duration vs. Num Parallel Runs')\n",
    "    axes[2].set_xlabel('Number of Parallel Runs')\n",
    "    axes[2].set_ylabel('Overall Duration (seconds)')\n",
    "    axes[2].grid(True)\n",
    "    axes[2].set_xticks(num_parallel_runs_values)\n",
    "    \n",
    "    plt.tight_layout(rect=[0, 0.03, 1, 0.95])\n",
    "    plt.show()\n",
    "\n",
    "else:\n",
    "    print(f\"No data for num_parallel_runs trend analysis with N={N_values_to_test[0]}, p={fixed_p_plot}, max_iter={fixed_max_iter_plot}, trans_lvl={fixed_trans_lvl_plot}.\")\n",
    "\n",
    "# --- Trend: Best Energy, AR, Duration vs. 'transpilation_level' (fixed N, p, max_iter, num_runs) ---\n",
    "print(f\"\\n--- Trends vs. 'transpilation_level' (Fixed: N={N_values_to_test[0]}, p={fixed_p_plot}, max_iter={fixed_max_iter_plot}, num_runs={fixed_num_runs_plot}) ---\")\n",
    "df_filtered_transpile_trend = results_df[\n",
    "    (results_df['N'] == N_values_to_test[0]) &\n",
    "    (results_df['p'] == fixed_p_plot) &\n",
    "    (results_df['max_iterations_optimizer'] == fixed_max_iter_plot) &\n",
    "    (results_df['num_parallel_runs'] == fixed_num_runs_plot)\n",
    "]\n",
    "\n",
    "if not df_filtered_transpile_trend.empty:\n",
    "    fig, axes = plt.subplots(1, 3, figsize=(18, 5))\n",
    "    fig.suptitle(f'N={N_values_to_test[0]}, p={fixed_p_plot}, Max Iterations={fixed_max_iter_plot}, Runs={fixed_num_runs_plot}, Shots={num_shots_simulator_fixed}', fontsize=16)\n",
    "\n",
    "    axes[0].plot(df_filtered_transpile_trend['transpilation_level'], df_filtered_transpile_trend['best_overall_energy'], marker='o', linestyle='-', color='blue')\n",
    "    axes[0].set_title('Best Overall Energy vs. Transpilation Level')\n",
    "    axes[0].set_xlabel('Transpilation Level')\n",
    "    axes[0].set_ylabel('Energy')\n",
    "    axes[0].grid(True)\n",
    "    axes[0].set_xticks(transpilation_levels_to_test)\n",
    "\n",
    "    if not df_filtered_transpile_trend['approximation_ratio'].isnull().all():\n",
    "        axes[1].plot(df_filtered_transpile_trend['transpilation_level'], df_filtered_transpile_trend['approximation_ratio'], marker='o', linestyle='-', color='red')\n",
    "        axes[1].set_title('Approximation Ratio vs. Transpilation Level')\n",
    "        axes[1].set_xlabel('Transpilation Level')\n",
    "        axes[1].set_ylabel('Approximation Ratio (AR)')\n",
    "        axes[1].grid(True)\n",
    "        axes[1].axhline(y=0, color='gray', linestyle='--', linewidth=0.8, label='Optimal AR = 0')\n",
    "        axes[1].legend()\n",
    "    else:\n",
    "        axes[1].text(0.5, 0.5, \"AR not available\", horizontalalignment='center', verticalalignment='center', transform=axes[1].transAxes, fontsize=12)\n",
    "        axes[1].set_title('Approximation Ratio vs. Transpilation Level')\n",
    "        axes[1].set_xlabel('Transpilation Level')\n",
    "        axes[1].set_ylabel('Approximation Ratio (AR)')\n",
    "    axes[1].set_xticks(transpilation_levels_to_test)\n",
    "\n",
    "    axes[2].plot(df_filtered_transpile_trend['transpilation_level'], df_filtered_transpile_trend['overall_duration_seconds'], marker='o', linestyle='-', color='green')\n",
    "    axes[2].set_title('Overall Duration vs. Transpilation Level')\n",
    "    axes[2].set_xlabel('Transpilation Level')\n",
    "    axes[2].set_ylabel('Overall Duration (seconds)')\n",
    "    axes[2].grid(True)\n",
    "    axes[2].set_xticks(transpilation_levels_to_test)\n",
    "    \n",
    "    plt.tight_layout(rect=[0, 0.03, 1, 0.95])\n",
    "    plt.show()\n",
    "\n",
    "else:\n",
    "    print(f\"No data for transpilation_level trend analysis with N={N_values_to_test[0]}, p={fixed_p_plot}, max_iter={fixed_max_iter_plot}, num_runs={fixed_num_runs_plot}.\")\n",
    "\n",
    "\n",
    "print(\"\\nBenchmarking complete. Analyze the tables and plots above for insights into QAOA performance.\")"
   ],
   "outputs": [],
   "execution_count": null
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.7"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}